In [1]:
import re
import pandas as pd
import numpy as np

# 产品上架

In [15]:
# 归属人名称处理

In [16]:
shelves = pd.read_excel('产品上架.xls')

In [17]:
shelves = shelves[(shelves['产品类型'] == '标准品')&(shelves['产品状态'].isin(['审核通过', '审核中']))]

In [18]:
def split_num(x):
    if x:
        regex = '([0-9]+-?[0-9]+)'
        result = re.compile(regex).search(x).group()
        if result:
            return result
        else:
            return x

In [19]:
shelves['编码拆分'] = shelves['产品编码'].apply(split_num)

In [20]:
# 产品分类拆分
def divide_type(x):
    if x != np.nan:
        regex_three = '(.*?)/(.*?)/(.*)'
        regex_two = '(.*?)/(.*)'   # 非贪婪模式，/后的内容会取不到
        result_three = re.compile(regex_three, re.S).search(x)
        result_two = re.compile(regex_two, re.S).search(x)
        if result_three:
            class_one = result_three.group(1)
            class_two = result_three.group(2)
            class_three = result_three.group(3)
            return class_one,class_two,class_three
        if result_two:
            class_one = result_two.group(1)
            class_two = result_two.group(2)
            class_three = result_two.group(2)
            return class_one,class_two,class_three
        if '/' not in x:
            class_one = x
            class_two = x
            class_three = x
            return class_one,class_two,class_three
    return x

In [21]:
shelves['二级分类'] = shelves['产品类别'].apply(lambda x: divide_type(x)[1])
shelves['二级分类'] = shelves['二级分类'].apply(lambda x: x.replace(' ', ''))

In [22]:
people_list = pd.read_excel('E:/code/pbi_data/业务员.xlsx')

shelves = pd.merge(shelves, people_list[['业务员', '英文名']], left_on = '归属人', right_on = '业务员' , how='left')

In [23]:
product_line = pd.read_excel('E:/code/pbi_data/产线归属.xlsx')

In [24]:
shelves = pd.merge(shelves, product_line[['产品线2_0', '产品线2', '产品线归属']], left_on = '产线', right_on = '产品线2_0' , how='left')

In [25]:
shelves['产品名称'] = shelves['中文名称']
shelves['提交日期'] = shelves['创建时间']
shelves['负责业务'] = shelves['英文名']
shelves['状态'] = ''
shelves['当前状态'] = ''

In [26]:
shelves['日期'] = '2022/6/1'

In [27]:
header = ['日期', '产品编码', '产品名称', '状态','当前状态', '负责业务', '提交日期', '产品类别', '产品状态']
shelves_input = shelves[header]

shelves_input.to_excel('产品进度表.xlsx', index=False)

In [28]:
header_match = pd.read_excel('./辅助数据/匹配表表头.xlsx')
shelves_match = shelves[list(header_match)]

In [29]:
shelves_match.to_excel('产品匹配表.xlsx', index=False)

# 产品上传更新

In [30]:
upload_data = pd.read_excel('上传更新.xlsx', sheet_name = '上传', header = 1)
update_data = pd.read_excel('上传更新.xlsx', sheet_name = '更新', header = 1)

In [31]:
upload_data

,Office,业务,SKU,平台,通知日期,完成日期,是否新品,上传人
0,HF,Rena,301450,dc-rp+esp-rg+sage,6.22,6.22,是,Ivy
1,HF,Meggie,300957,dc-rp+esp-rg+sage,6.22,6.22,是,Ivy


In [32]:
update_data

,Office,Sales,SKU,Update,平台,通知日期,完成日期,更新人
0,HF,Jenna,201329,描述，关键词，图片,dc-rp+sage,6.20,6.20,Vincent
1,HF,Rena,124236,价格,dc-rg+esp-rg+sage,6.20,6.20,Vincent
2,HF,Jenna,300584,Large Size: Extra $1.7/pc for 3XL to 5XL,dc-rp+esp-rg+sage,6.20,6.20,Vincent
3,HF,Jenna,300583,Milk Silk Material: Free,dc-rp+esp-rg+sage,6.20,6.20,Vincent
4,HF,Jenna,300585,Milk Silk Material: Free,dc-rp+esp-rg+sage,6.20,6.20,Vincent
...,...,...,...,...,...,...,...,...
190,HF,Rena,204830,主图，价格,dc-rp+esp-rg+sage,6.24,6.24,Ivy
191,HF,Rena,204830-Ocean,主图,dc-rp+esp-rg+sage,6.24,6.24,Ivy
192,HF,Rena,204109,"价格,交期，数量",dc-rp+esp-rg+sage,6.24,6.24,Ivy
193,YW,Funny,204362,品名、关键词、价格,dc-rp+sage,6.20,6.20,Coco


In [33]:
district = {'HF':'合肥', 'SZ':'苏州', 'YW':'义乌'}

upload_data['地区'] = upload_data['Office'].map(district)
update_data['地区'] = update_data['Office'].map(district)

In [34]:
upload_data['辅助'] = upload_data['业务']
update_data['辅助'] = update_data['Sales']

<b><font size=2 color=green>时间字段手动填写：</font></b>

In [35]:
upload_data['年份'] = update_data['年份'] = '2022'
upload_data['月份'] = update_data['月份'] = '6'
upload_data['年月'] = update_data['年月'] = '2022/6/1'
upload_data['周'] = update_data['周'] = '第4周'

month = '6.'
new_month = '2022/6/'

In [36]:
upload_data['通知日期'] = upload_data['通知日期'].apply(lambda x: str(x).replace(month, new_month))
upload_data['完成日期'] = upload_data['完成日期'].apply(lambda x: str(x).replace(month, new_month))

update_data['通知日期'] = update_data['通知日期'].apply(lambda x: str(x).replace(month, new_month))
update_data['完成日期'] = update_data['完成日期'].apply(lambda x: str(x).replace(month, new_month))

In [37]:
staff_list = pd.read_excel('E:/code/pbi_data/业务员.xlsx')

In [38]:
upload_data = pd.merge(upload_data, staff_list[['名', '英文名']], left_on = '辅助', right_on = '名', how = 'left')

In [39]:
update_data = pd.merge(update_data, staff_list[['名', '英文名']], left_on = '辅助', right_on = '名', how = 'left')

In [40]:
upload_data['编码拆分'] = upload_data['SKU'].astype(str)
upload_data['编码拆分'] = upload_data['编码拆分'].apply(split_num)
upload_data = pd.merge(upload_data, shelves[['编码拆分', '中文名称', '二级分类', '产品线2', '产品线归属']], on = '编码拆分', how = 'left')

In [41]:
update_data['编码拆分'] = update_data['SKU'].astype(str)
update_data['编码拆分'] = update_data['编码拆分'].apply(split_num)
update_data = pd.merge(update_data, shelves[['编码拆分', '中文名称', '二级分类', '产品线2', '产品线归属']], on = '编码拆分', how = 'left')

In [42]:
# sage中用到的匹配表
upload_data['注释'] = upload_data['业务组'] = upload_data['是否产线2'] = upload_data['备注'] = ''
match_table = upload_data[['SKU', '编码拆分', '注释', '备注', '完成日期', '英文名', '业务组', '中文名称', '产品线2', '是否产线2', '产品线2', '产品线归属']]
match_table.to_excel('匹配0102.xlsx', index=False)

In [43]:
upload_data['分组'] = ''
upload_data.rename(columns={'SKU':'上传编码', '平台':'上传平台', '通知日期':'通知时间', '完成日期':'完成时间', '英文名':'人员', '中文名称':'上传产品', '二级分类':'上传类目', '产品线2':'产品线'}, inplace=True)

In [44]:
update_data['分组'] = ''
update_data.rename(columns={'SKU':'更新编码', 'Update':'修改内容', '通知日期':'通知时间', '完成日期':'完成时间', '英文名':'人员', '中文名称':'更新产品', '二级分类':'更新类目', '产品线2':'产品线'}, inplace=True)

In [45]:
upload_header = pd.read_excel('./辅助数据/上传更新表头.xlsx', sheet_name='上传')
update_header = pd.read_excel('./辅助数据/上传更新表头.xlsx', sheet_name='更新')

In [46]:
upload_data = upload_data[list(upload_header)]

In [47]:
update_data = update_data[list(update_header)]

In [48]:
upload_data.to_excel('上传表维护结果.xlsx', index=False)

In [49]:
update_data.to_excel('更新表维护结果.xlsx', index=False)

# 询盘

In [52]:
inquiry_output_data = pd.read_excel('询盘导出表.xlsx')

D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [53]:
inquiry_output_data

,Date,Sales,Remarks,Inquiry #,RP,ESP/SAGE/DC/Website/Other,Item NO.,Item Name,Quantity,Email,Contact Name,Company Name,Tel.,Fax,Website,Add.
0,2022-06-20,Terry Chu,P22052676询价下单,PO# 7372 null,P,Email,202452,Mouse Pad,1000,Gloriabrayer@comcast.net,NaN,Iconic Promotions,954-385-4771,NaN,NaN,NaN
1,2022-06-20,Clover Wang,Email下单,PO#40308 null,P,Email,120888,"10"" Folding Flyer With Pouch",500,jill@mprintspromo.com,NaN,M'Prints Promtional Products,601.553.1522,NaN,NaN,NaN
2,2022-06-20,Meggie Fan,NaN,P22061906,P,Email,129114,Sports Drawstring Backpack with Zipper Pocket,1100,jay@universitysilkscreen.com,NaN,University Silkscreen,405-366-8256,NaN,NaN,NaN
3,2022-06-20,Jane Zhou,NaN,P22061907,P,Email,NaN,15 oz photo mugs,144-288-500,mail@wlcpromotions.com/\t wlc.promotionalmarke...,NaN,WLC Promotions,562-443-1367,NaN,NaN,NaN
4,2022-06-20,Tori Lee,客户直接发PO,PO 18135 null,P,Email,202242,Flip Open PU Leather Luggage Tag,300,cindy.beierle@ideasource.ca,NaN,Ideasource Recognition & Rewards Inc.,403-912-9161,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,2022-06-25,Karen Li,客户直接联系,P22062572,P,Email,123915,Drawstring bag/bottle/Single Pocket Coin Pouch,1600,ara@raptclothing.com,NaN,Rapt Clothing Inc,323-697-3733,NaN,NaN,NaN
722,2022-06-25,Selina Gong,客户直接联系,P22062573,P,Email,NaN,drawstring bag,/,sales@fxpromowear.com,NaN,Fxpromowear,206-383-1861,NaN,NaN,NaN
723,2022-06-25,Daniel Chen,客户直接联系,P22062574,P,Email,202097,Lanyard,/,Robyn@abracadabraprinting.com,NaN,Abracadabra Printing,206-3473-9087,NaN,NaN,NaN
724,2022-06-25,Clover Wang,报价下单,PO WHSKBEANIES062422 null,P,Email,NaN,BLACK WOOL BEANIE CAP,50,rok@warmachinemerch.com,NaN,War Machine Marketing,310-991-2800,NaN,NaN,NaN


In [54]:
inquiry_output_data.rename(columns={
    'Date': '日期',
    'Sales': '业务员0',
    'Inquiry #': '询盘号',
    'ESP/SAGE/DC/Website/Other': '询盘来源',
    'Item NO.': '产品编号',
    'Item Name': '产品名称',
    'Quantity': '询盘数量（PCS）',
    'Email': '客户邮箱',
    'Contact Name': '客户名',
    'Company Name': '客户公司名',
    'Tel.': '客户电话'
},
                           inplace=True)

In [55]:
inquiry_output_data['年份'] = '2022'
inquiry_output_data['月份'] = '6'
inquiry_output_data['年月'] = '2022/6/1'
inquiry_output_data['周'] = '第4周'

In [56]:
inquiry_man = pd.read_excel('./辅助数据/询盘表业务员.xlsx')

In [57]:
inquiry_output_data = pd.merge(inquiry_output_data, inquiry_man, on='业务员0', how='left')

In [58]:
inquiry_header = pd.read_excel('./辅助数据/询盘表头.xlsx')

In [59]:
inquiry_output_data['是否新客户'] = inquiry_output_data[
    '二级产品类目'] = inquiry_output_data['产品线'] = inquiry_output_data[
        '产品类型'] = inquiry_output_data['$单价'] = inquiry_output_data[
            '成本价（￥）'] = inquiry_output_data[
                '询盘状态（是否下单）'] = inquiry_output_data['PI'] = ''

In [60]:
inquiry_output_data = inquiry_output_data[list(inquiry_header)]

In [61]:
inquiry_output_data = inquiry_output_data[inquiry_output_data['业务员'].notnull()]

In [62]:
inquiry_output_data.to_excel('询盘导出表维护结果.xlsx', index=False)